In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


import geopandas as gpd
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging
from pykrige.uk import UniversalKriging
from shapely.geometry import Point, Polygon
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import descartes
import math
import matplotlib.colors as mcolors
import pyproj
import folium

# LIBRERIAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


import geopandas as gpd
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging
from pykrige.uk import UniversalKriging
from shapely.geometry import Point, Polygon
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import descartes
import math
import matplotlib.colors as mcolors
import pyproj
import folium

import geopandas as gpd
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging
from pykrige.uk import UniversalKriging
from shapely.geometry import Point, Polygon
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import descartes
import math
import matplotlib.colors as mcolors
import pyproj
import folium
import contextily as ctx
from shapely.geometry import box
import xarray as xr
import matplotlib.animation as animation
import os
from branca.colormap import linear
from shapely.wkt import loads
from scipy.spatial import cKDTree
from sklearn.ensemble import RandomForestRegressor

In [18]:
wdir = "../Dades/AMS_Observacions/"
crs_latlon = 'EPSG:4326'  # WGS84
crs_utm = "EPSG:32631"

In [19]:
df_no2 = pd.read_csv('../Dades/AMS_Observacions/gene_sconcno2_2023_xvpca_emep_port.csv') 
df_estacions = pd.read_csv('../Dades/AMS_Observacions/XVPCA_info_sconcno2_2023.csv')


# Convert 'Date' column to datetime format
df_no2['Date'] = pd.to_datetime(df_no2['Date'])

# Pivot the dataframe to have each station ID as a column and the Date as the index
# columnas_es = [col for col in df_no2.columns if col.startswith('ES')]
columnas_es =df_no2.drop(['Date'], axis=1)


# Transpose the dataframe to have each station ID as a row and the Date as the column
df_no2_transposed = df_no2.set_index('Date').transpose().reset_index()

# Melt the dataframe to have a 'Value' column for each hour
df_no2_melted = pd.melt(df_no2_transposed, id_vars=['index'], var_name='Date', value_name='no2')

# Rename the 'index' column to 'Station'
df_no2_melted.rename(columns={'index': 'Station'}, inplace=True)

# Drop rows with NaN values
df_no2_melted.dropna(subset=['no2'], inplace=True)

print(df_no2_melted)

# Join the melted dataframe with the station information dataframe
df_obs = pd.merge(df_no2_melted, df_estacions, left_on='Station', right_on='code', how='inner')

# Drop the 'code' column as it is redundant
df_obs.drop(columns=['code','type'], inplace=True)
df_obs['geometry'] = gpd.points_from_xy(df_obs['lon'], df_obs['lat'], crs=crs_utm)
print(df_obs)

        Station                 Date   no2
2       ES1480A  2022-12-31 23:00:00  62.0
3       ES1396A  2022-12-31 23:00:00  60.0
4       ES1992A  2022-12-31 23:00:00  45.0
5       ES0691A  2022-12-31 23:00:00  43.0
6       ES0692A  2022-12-31 23:00:00  45.0
...         ...                  ...   ...
560633  ES2017A  2023-12-31 22:00:00   7.0
560634  ES1930A  2023-12-31 22:00:00   1.0
560635  ES1948A  2023-12-31 22:00:00   3.0
560636  ES1855A  2023-12-31 22:00:00   2.0
560637  ES1854A  2023-12-31 22:00:00   1.0

[529219 rows x 3 columns]
        Station                 Date   no2        lat       lon  \
0       ES1480A  2022-12-31 23:00:00  62.0  41.398762  2.153472   
1       ES1396A  2022-12-31 23:00:00  60.0  41.378803  2.133098   
2       ES1992A  2022-12-31 23:00:00  45.0  41.387273  2.115661   
3       ES0691A  2022-12-31 23:00:00  43.0  41.403716  2.204736   
4       ES0692A  2022-12-31 23:00:00  45.0  41.370760  2.114771   
...         ...                  ...   ...        ...  

In [20]:
# df_caliope = pd.read_csv('../NO2_no_h.csv') 
df_caliope = pd.read_csv('../Dades/NO2.csv') 

df_caliope.rename(columns={'sconcno2': 'no2', 'time': 'Date'}, inplace=True)

df_caliope['geometry'] = gpd.points_from_xy(df_caliope['lon'], df_caliope['lat'], crs=crs_utm)

df_combined = pd.concat([df_obs, df_caliope], ignore_index=True)

# Convert df_combined to a GeoDataFrame
df_combined = gpd.GeoDataFrame(df_combined, geometry='geometry', crs=crs_utm)

# Display the first few rows to verify the conversion
df_combined.head()


,Station,Date,no2,lat,lon,geometry
0,ES1480A,2022-12-31 23:00:00,62.0,41.398762,2.153472,POINT (2.153 41.399)
1,ES1396A,2022-12-31 23:00:00,60.0,41.378803,2.133098,POINT (2.133 41.379)
2,ES1992A,2022-12-31 23:00:00,45.0,41.387273,2.115661,POINT (2.116 41.387)
3,ES0691A,2022-12-31 23:00:00,43.0,41.403716,2.204736,POINT (2.205 41.404)
4,ES0692A,2022-12-31 23:00:00,45.0,41.370760,2.114771,POINT (2.115 41.371)


In [21]:
# Agrupar por 'geometry' y calcular la media de 'no2'
df_combined_mean = df_caliope.groupby('geometry')['no2'].mean().reset_index()

# Mostrar el resultado
print("Max NO2:", df_combined_mean['no2'].max())
print("Min NO2:", df_combined_mean['no2'].min())

Max NO2: 0.044042407205128205
Min NO2: 0.00010403053067096775


In [ ]:
# domain_polygon = gpd.read_file("../Dades/Hands-on_Downscaling/Data_python/def_domini_bcn.geojson")

domain_polygon= gpd.read_file("../Dades/Hands-on_Downscaling/Data_python/def_domini_tar.geojson")

minx, miny, maxx, maxy = domain_polygon.total_bounds

# Crear la cuadrícula de 150m x 150m
cell_size = 500  # Tamaño de cada celda en metros
x_coords = np.arange(minx, maxx, cell_size)
y_coords = np.arange(miny, maxy, cell_size)

grid_cells = [box(x, y, x + cell_size, y + cell_size) for x in x_coords for y in y_coords]
grid = gpd.GeoDataFrame(grid_cells, columns=["geometry"], crs=crs_utm)

# Recortar la cuadrícula al área de Barcelona
grid_bcn = gpd.overlay(grid, domain_polygon, how="intersection")

# Convertir a WGS84 para usar en Folium
grid_bcn_wgs84 = grid_bcn.to_crs(epsg=4326)

# Calcular los puntos centrales de las celdas (centroides)
grid_bcn_wgs84["centroid"] = grid_bcn_wgs84.geometry.centroid

In [23]:
# Crear un nuevo GeoDataFrame a partir de grid_bcn_wgs84["geometry"]
grid_gdf = gpd.GeoDataFrame(geometry=grid_bcn_wgs84["centroid"], crs=crs_latlon)
centroid_list = list(grid_gdf["geometry"])
# Mostrar las primeras filas
print(centroid_list)

[<POINT (0.771 41.6)>, <POINT (0.771 41.604)>, <POINT (0.771 41.609)>, <POINT (0.772 41.613)>, <POINT (0.772 41.618)>, <POINT (0.772 41.622)>, <POINT (0.772 41.627)>, <POINT (0.772 41.631)>, <POINT (0.772 41.636)>, <POINT (0.772 41.64)>, <POINT (0.772 41.643)>, <POINT (0.777 41.6)>, <POINT (0.777 41.605)>, <POINT (0.777 41.609)>, <POINT (0.777 41.614)>, <POINT (0.777 41.618)>, <POINT (0.776 41.623)>, <POINT (0.776 41.627)>, <POINT (0.776 41.632)>, <POINT (0.776 41.636)>, <POINT (0.776 41.641)>, <POINT (0.774 41.643)>, <POINT (0.783 41.6)>, <POINT (0.783 41.605)>, <POINT (0.783 41.609)>, <POINT (0.783 41.614)>, <POINT (0.783 41.618)>, <POINT (0.782 41.623)>, <POINT (0.782 41.627)>, <POINT (0.782 41.632)>, <POINT (0.782 41.636)>, <POINT (0.782 41.641)>, <POINT (0.789 41.601)>, <POINT (0.789 41.605)>, <POINT (0.789 41.609)>, <POINT (0.789 41.614)>, <POINT (0.789 41.618)>, <POINT (0.788 41.623)>, <POINT (0.788 41.627)>, <POINT (0.788 41.632)>, <POINT (0.788 41.636)>, <POINT (0.788 41.641)>

In [24]:
from scipy.spatial import cKDTree
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

stations_gdf = gpd.GeoDataFrame(df_obs.drop_duplicates(subset=['geometry']).copy(), geometry='geometry')
# Crear GeoDataFrame con puntos de predicción
obs_locations = gpd.GeoDataFrame(
    {'geometry': centroid_list},
    crs=crs_utm
)


# Extraer coordenadas y valores del dataset conocido
coords = np.array([(geom.x, geom.y) for geom in df_combined_mean.geometry])
values = df_combined_mean['no2'].values

# Crear el árbol KD
tree = cKDTree(coords)

# Función IDW
def idw(x, y, tree, coords, values, power=2):
    """
    Interpolación por inverso de la distancia (IDW).
    - x, y: Coordenadas de predicción
    - tree: KDTree construido con los datos conocidos
    - coords: Coordenadas conocidas
    - values: Valores conocidos
    - power: Peso de la distancia
    """
    # Encontrar distancias y vecinos
    distances, idx = tree.query(np.c_[x, y], k=len(coords))
    
    # Manejo de distancias 0 (puntos coincidentes)
    distances[distances == 0] = 1e-10  # Evitar división por cero
    
    # Calcular pesos inversos a las distancias
    weights = 1 / distances**power
    
    # Normalizar los pesos
    weights /= np.sum(weights, axis=1, keepdims=True)
    
    # Calcular valores interpolados
    interpolated_values = np.sum(weights * values[idx], axis=1)
    return interpolated_values

# Coordenadas de los puntos a predecir
prediction_coords = np.array([(geom.x, geom.y) for geom in obs_locations.geometry])
x_pred, y_pred = prediction_coords[:, 0], prediction_coords[:, 1]

# Aplicar IDW a los puntos
obs_locations['predicted_value'] = idw(x_pred, y_pred, tree, coords, values)

print(obs_locations)

resultats_station = obs_locations.copy()

# Coordenadas de los puntos de las estaciones
station_coords = np.array([(geom.x, geom.y) for geom in stations_gdf.geometry])
x_station, y_station = station_coords[:, 0], station_coords[:, 1]

# Aplicar IDW a los puntos de las estaciones
stations_gdf['predicted_value'] = idw(x_station, y_station, tree, coords, values)

print(stations_gdf)
print(stations_gdf.head())


                 geometry  predicted_value
0      POINT (0.771 41.6)         0.004586
1    POINT (0.771 41.604)         0.004578
2    POINT (0.771 41.609)         0.004597
3    POINT (0.772 41.613)         0.004603
4    POINT (0.772 41.618)         0.004641
..                    ...              ...
134   POINT (0.846 41.61)         0.004887
135  POINT (0.846 41.615)         0.004848
136  POINT (0.846 41.619)         0.004858
137  POINT (0.846 41.624)         0.004915
138  POINT (0.845 41.627)         0.004949

[139 rows x 2 columns]
       Station                 Date   no2        lat       lon  \
0      ES1480A  2022-12-31 23:00:00  62.0  41.398762  2.153472   
1      ES1396A  2022-12-31 23:00:00  60.0  41.378803  2.133098   
2      ES1992A  2022-12-31 23:00:00  45.0  41.387273  2.115661   
3      ES0691A  2022-12-31 23:00:00  43.0  41.403716  2.204736   
4      ES0692A  2022-12-31 23:00:00  45.0  41.370760  2.114771   
...        ...                  ...   ...        ...       ...  

In [25]:
# Cargar el shapefile
archivo_shapefile = "../Dades/OpenTransportMap/Barcelona/roadlinks_ES511.shp"
gdf_barcelona = gpd.read_file(archivo_shapefile)
gdf_girona = gpd.read_file("../Dades/OpenTransportMap/Girona/roadlinks_ES512.shp")
gdf_lleida = gpd.read_file("../Dades/OpenTransportMap/Lleida/roadlinks_ES513.shp")
gdf_tarragona = gpd.read_file("../Dades/OpenTransportMap/Tarragona/roadlinks_ES514.shp")

# Concatenate the GeoDataFrames
gdf = pd.concat([gdf_barcelona, gdf_girona, gdf_lleida, gdf_tarragona], ignore_index=True)
# Mostrar las primeras filas de la tabla de atributos
#print(gdf['functional'].head())

# Obtener los tipos únicos de la columna 'functional'
tipos_functional = gdf['functional'].unique()

# Asignar valores de prioridad a las categorías
valores_prioridad = {
    'mainRoad': 4 / 4,
    'firstClass': 3 / 4,
    'secondClass': 2 / 4,
    'thirdClass': 1 / 4,
    'fourthClass': 0,
    'fifthClass': 0,    
}

# Crear una nueva columna en el GeoDataFrame con los valores de prioridad
gdf['prioridad'] = gdf['functional'].map(valores_prioridad)

# Crear un nuevo DataFrame con las columnas 'sourceid', 'functional' y 'prioridad'
df_transport = gdf[['sourceid', 'prioridad','geometry']].drop_duplicates().reset_index(drop=True)
# Eliminar filas con valores de prioridad igual a 0
df_transport = df_transport[df_transport['prioridad'] != 0]
print(df_transport.head())

# Guardar el GeoDataFrame en un archivo CSV
#gdf.to_csv("./carreteres.csv", index=False)

     sourceid  prioridad                                           geometry
8   247323996       0.25  LINESTRING (2.229 41.66569, 2.229 41.66606, 2....
10  199276193       0.25  LINESTRING (1.91614 41.5961, 1.91615 41.59607,...
11  199276720       1.00    LINESTRING (1.63758 41.73453, 1.63799 41.73411)
13  199316496       1.00  LINESTRING (1.92758 41.79895, 1.92656 41.7982,...
14  199333176       1.00  LINESTRING (2.16105 41.93252, 2.16175 41.93245...


In [26]:
from shapely.geometry import LineString
from shapely.ops import nearest_points

roads_gdf = gpd.GeoDataFrame(df_transport.copy(), geometry='geometry')

stations_gdf = stations_gdf[stations_gdf.geometry.is_valid]
roads_gdf = roads_gdf[roads_gdf.geometry.is_valid]

def encontrar_carretera_cercana(punto):
    # Busca los índices de las carreteras cercanas
    posibles_indices = list(roads_gdf.sindex.intersection(punto.buffer(0.01).bounds))
    
    # Si hay carreteras cercanas
    if posibles_indices:
        # Filtrar y encontrar la más cercana
        carreteras_cercanas = roads_gdf.iloc[posibles_indices]
        distancias = carreteras_cercanas.geometry.distance(punto)
        indice_mas_cercano = distancias.idxmin()
        road = roads_gdf.loc[indice_mas_cercano]
        return road['prioridad']
    
    return None

In [27]:
# Aplicar la función a cada punto
import warnings

# Aplicar la función a cada punto en stations_gdf
warnings.filterwarnings('ignore')

stations_gdf['carretera_cercana'] = stations_gdf['geometry'].apply(encontrar_carretera_cercana)

# Mostrar los resultados
print(stations_gdf[['geometry', 'carretera_cercana','no2']].head())



               geometry  carretera_cercana   no2
0  POINT (2.153 41.399)               0.75  62.0
1  POINT (2.133 41.379)               0.25  60.0
2  POINT (2.116 41.387)               0.25  45.0
3  POINT (2.205 41.404)               0.25  43.0
4  POINT (2.115 41.371)               0.25  45.0


In [28]:
def encontrar_road_dict(punto, carreteras_gdf, buffer_distance=0.01):
    # Crear un buffer alrededor del punto
    punto_buffer = punto.buffer(buffer_distance)
    
    # Encontrar carreteras dentro del buffer
    road_dict = carreteras_gdf[carreteras_gdf.intersects(punto_buffer)]
    
    # Si no hay carreteras cercanas, devolver None
    if road_dict.empty:
        return None
    
    # Calcular distancias y otra información
    road_dict['distancia'] = road_dict.geometry.apply(lambda geom: geom.distance(punto))
    
    # Ordenar por distancia
    road_dict = road_dict.sort_values('distancia')
    
    # Extraer información relevante
    # Eliminar duplicados basados en la distancia
    road_dict = road_dict.drop_duplicates(subset=['distancia']).head(10)
    
    info_carreteras = road_dict.apply(lambda row: {
        'sourceid': row['sourceid'],
        'prioridad': row['prioridad'],
        'distancia': row['distancia']
    }, axis=1).tolist()
    
    return info_carreteras

# Aplicar la función a cada punto
def procesar_estaciones(estaciones_gdf, buffer_size=0.05):
    # Crear una copia para no modificar el original
    estaciones_procesadas = estaciones_gdf.copy()
    
    # Aplicar la función de búsqueda de carreteras cercanas
    estaciones_procesadas['road_dict'] = estaciones_procesadas.geometry.apply(
        lambda punto: encontrar_road_dict(punto, roads_gdf, buffer_distance=buffer_size)
    )
    
    return estaciones_procesadas

# Ejecutar el procesamiento
stations_gdf = procesar_estaciones(stations_gdf)

resultats_station = procesar_estaciones(resultats_station)

print(resultats_station['road_dict'].iloc[0])
print(stations_gdf.head())
print(stations_gdf['road_dict'].iloc[0])

print(stations_gdf['no2'])

[{'sourceid': 180730885, 'prioridad': 0.75, 'distancia': 0.026638156485719298}, {'sourceid': 79925033, 'prioridad': 0.75, 'distancia': 0.026982520179300876}, {'sourceid': 176055500, 'prioridad': 0.75, 'distancia': 0.027177847085295036}, {'sourceid': 176055516, 'prioridad': 0.25, 'distancia': 0.027293488845415517}, {'sourceid': 176055502, 'prioridad': 0.25, 'distancia': 0.02745811885734562}, {'sourceid': 176055512, 'prioridad': 0.75, 'distancia': 0.02757673801345167}, {'sourceid': 176055524, 'prioridad': 0.25, 'distancia': 0.027662382860811772}, {'sourceid': 176055505, 'prioridad': 0.25, 'distancia': 0.027687244695620557}, {'sourceid': 176055522, 'prioridad': 0.25, 'distancia': 0.02770891107356914}, {'sourceid': 327566600, 'prioridad': 0.25, 'distancia': 0.027737099320428043}]
   Station                 Date   no2        lat       lon  \
0  ES1480A  2022-12-31 23:00:00  62.0  41.398762  2.153472   
1  ES1396A  2022-12-31 23:00:00  60.0  41.378803  2.133098   
2  ES1992A  2022-12-31 23:0

In [29]:

df_station = stations_gdf[['geometry', 'predicted_value', 'road_dict', 'no2']].copy()
df_station.head()

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Preparar datos para el modelo
def extraer_caracteristicas(road_dict):
    if not road_dict or len(road_dict) == 0:
        return {
            'num_carreteras': 0,
            'prioridad_media': 0,
            'prioridad_min': 0,
            'prioridad_max': 0,
            'distancia_media': 0,
            'distancia_min': 0
        }
    
    # Extraer información de las carreteras
    num_carreteras = len(road_dict)
    prioridades = [road['prioridad'] for road in road_dict]
    distancias = [road['distancia'] for road in road_dict]
    
    return {
        'num_carreteras': num_carreteras,
        'prioridad_media': np.mean(prioridades),
        'prioridad_min': np.min(prioridades),
        'prioridad_max': np.max(prioridades),
        'distancia_media': np.mean(distancias),
        'distancia_min': np.min(distancias)
    }

# Preparar el DataFrame de características
def preparar_datos(df_station):
    # Extraer características de las carreteras
    
    caracteristicas_carreteras = df_station['road_dict'].apply(extraer_caracteristicas)
    
    # Convertir a DataFrame
    df_caracteristicas = pd.DataFrame(caracteristicas_carreteras.tolist())
    
    # Añadir la columna de NO2
    if 'no2' in df_station.columns:
        df_caracteristicas['no2'] = df_station['no2']
    else:
        df_caracteristicas['geometry'] = df_station['geometry']
    df_caracteristicas['predicted_value'] = df_station['predicted_value']
    
    return df_caracteristicas

dades_forest = preparar_datos(df_station).drop(columns=['num_carreteras'])
resultat_categoriques = preparar_datos(resultats_station).drop(columns=['num_carreteras'])

print(dades_forest.head())
print(resultat_categoriques.head())

   prioridad_media  prioridad_min  prioridad_max  distancia_media  \
0             0.75           0.75           0.75         0.001016   
1             0.35           0.25           0.75         0.002787   
2             0.40           0.25           0.75         0.001980   
3             0.25           0.25           0.25         0.001888   
4             0.25           0.25           0.25         0.001177   

   distancia_min   no2  predicted_value  
0       0.000304  62.0         0.016475  
1       0.002102  60.0         0.022966  
2       0.001486  45.0         0.014173  
3       0.001422  43.0         0.018538  
4       0.000299  45.0         0.016984  
   prioridad_media  prioridad_min  prioridad_max  distancia_media  \
0             0.45           0.25           0.75         0.027392   
1             0.25           0.25           0.25         0.025574   
2             0.25           0.25           0.25         0.021220   
3             0.25           0.25           0.25         

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings

# Asegúrate de que no hay NaN en los datos
# Definir las características y la variable objetivo
features = [ 'prioridad_media', 'prioridad_min', 'prioridad_max', 
            'distancia_media', 'distancia_min', 'predicted_value']
dades_forest = dades_forest.dropna(subset=features + ['no2'])

# Escalar los datos
scaler = StandardScaler()
X_features = dades_forest[features].copy()
print(X_features)
X_features[['prioridad_media', 'prioridad_min', 'prioridad_max', 'distancia_media', 'distancia_min']] = scaler.fit_transform(X_features[[ 'prioridad_media', 'prioridad_min', 'prioridad_max', 'distancia_media', 'distancia_min']])
y = dades_forest['no2']


resultat_categoriques[[ 'prioridad_media', 'prioridad_min', 'prioridad_max', 'distancia_media', 'distancia_min']] = scaler.transform(resultat_categoriques[[ 'prioridad_media', 'prioridad_min', 'prioridad_max', 'distancia_media', 'distancia_min']])


# Entrenar el modelo con todos los datos
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_features, y)

# Predecir y evaluar con los mismos datos de entrenamiento
y_pred_rf = rf_model.predict(X_features)

# Métricas de evaluación
mse_rf = mean_squared_error(y, y_pred_rf)
r2_rf = r2_score(y, y_pred_rf)

# Calcular RMSE
rmse_rf = np.sqrt(mse_rf)

# Imprimir resultados
print("Random Forest Mean Squared Error:", mse_rf)
print("Random Forest R² Score:", r2_rf)
print("Random Forest Root Mean Squared Error:", rmse_rf)

# Imprimir las predicciones
print("Predicciones:", y_pred_rf)


    prioridad_media  prioridad_min  prioridad_max  distancia_media  \
0             0.750           0.75           0.75         0.001016   
1             0.350           0.25           0.75         0.002787   
2             0.400           0.25           0.75         0.001980   
3             0.250           0.25           0.25         0.001888   
4             0.250           0.25           0.25         0.001177   
5             0.250           0.25           0.25         0.001952   
6             0.450           0.25           0.50         0.001742   
7             0.325           0.25           1.00         0.003858   
8             0.250           0.25           0.25         0.004328   
9             0.250           0.25           0.25         0.001499   
10            0.250           0.25           0.25         0.000659   
11            0.250           0.25           0.25         0.004448   
12            0.250           0.25           0.25         0.002150   
13            0.725 

In [31]:
resultat_categoriques.head()
# Realizar predicciones utilizando el modelo entrenado
predicciones = rf_model.predict(resultat_categoriques[features])

# Mostrar las predicciones
resultat_categoriques['final_prediction'] = predicciones
mod_obs_locations= resultat_categoriques.copy()

print(mod_obs_locations.columns)

Index(['prioridad_media', 'prioridad_min', 'prioridad_max', 'distancia_media',
       'distancia_min', 'geometry', 'predicted_value', 'final_prediction'],
      dtype='object')


In [ ]:
from IPython.display import display

# Crear un rango de colores basado en predicted_value
colormap = linear.viridis.scale(
    mod_obs_locations['final_prediction'].min(),
    mod_obs_locations['final_prediction'].max()
)
colormap.caption = "NO₂ Predicted Concentration"

# Crear un mapa centrado en las coordenadas promedio de los puntos
map_center = [
    mod_obs_locations.geometry.apply(lambda p: p.y).mean(),
    mod_obs_locations.geometry.apply(lambda p: p.x).mean()
]
m = folium.Map(location=map_center, zoom_start=12)

# Agregar los puntos al mapa
for _, row in mod_obs_locations.iterrows():
    color = colormap(row['final_prediction'])  # Obtener el color para el valor
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],  # Coordenadas del punto
        radius=5,  # Tamaño del marcador
        color=color,  # Color del borde
        fill=True,
        fill_color=color,  # Color del relleno
        fill_opacity=0.7,
        popup=f"NO₂: {row['final_prediction']:.2f}"  # Información desplegable
    ).add_to(m)

# Agregar la barra de colores al mapa
colormap.add_to(m)

# Mostrar el mapa
m
# Save the map to an HTML file
m.save("predicted_no2_map_carreteres_tar.html")

# Display the map in the notebook
display(m)